Doing basic analysis

In [120]:
from model import read_into_dataframe, extract_data
import matplotlib.pyplot as plt

df = read_into_dataframe("./data/bs140513_032310.csv")
df.info()
print(df.head())
print(df.nunique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 594643 entries, 0 to 594642
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   step         594643 non-null  int64  
 1   customer     594643 non-null  object 
 2   age          594643 non-null  object 
 3   gender       594643 non-null  object 
 4   zipcodeOri   594643 non-null  object 
 5   merchant     594643 non-null  object 
 6   zipMerchant  594643 non-null  object 
 7   category     594643 non-null  object 
 8   amount       594643 non-null  float64
 9   fraud        594643 non-null  int64  
dtypes: float64(1), int64(2), object(7)
memory usage: 45.4+ MB
   step       customer  age gender zipcodeOri       merchant zipMerchant  \
0     0  'C1093826151'  '4'    'M'    '28007'   'M348934600'     '28007'   
1     0   'C352968107'  '2'    'M'    '28007'   'M348934600'     '28007'   
2     0  'C2054744914'  '4'    'F'    '28007'  'M1823072687'     '28007'   
3     

Age should have almost 0 unique fields so we are looking at that.
We can see that there is one U which I am assuming is a  mistake and since it is only one row we will just remove it

In [124]:
print(df["age"].unique())
print(df["step"].min())
df["step"].max()

["'4'" "'2'" "'3'" "'5'" "'1'" "'6'" "'U'" "'0'"]
0


179

Interestingly there are a two other genders 'E' and 'U' but in much lower proportion then 'F' an 'M'.

In [ ]:
df["gender"].value_counts()

Looking at how even the data set is in terms of fraud and not fraud.
It is obvious that fraud and non fraud is imbalanced if scikitlearn does not automatically scramble data I should

In [ ]:
df["fraud"].value_counts()

In [ ]:
# just moved to different block so that it isn't rerun
x_df, y_df = extract_data(df)
df = x_df.join(y_df)

In [ ]:
print(df.corr())
df.describe().T

Looking at the one hot encoded columns which I created

In [ ]:
boolean_columns = df.select_dtypes(include=bool).columns
for column in boolean_columns:
    print(df[column].value_counts())